In [ ]:
# Find the average salaries of each job position(assuming salary is the total of all pays).
# Which job title has the highest full time employees?
# List the name of employees who work for the police department?
# Find the job titles along with the employees name and ids.
# Find the number of employees in each job title. 
# List out the names and positions of employees whose total pay is greater than 180000.
# List the names and ids of employees who have never done overtime.



In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import udf,col,lit

spark = SparkSession.builder.appName("Netflix_Title").getOrCreate()


22/10/21 11:49:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [17]:
salarydf = spark.read.json('Salaries.json')
salarydf.printSchema()

root
 |-- Agency: string (nullable = true)
 |-- BasePay: string (nullable = true)
 |-- Benefits: string (nullable = true)
 |-- EmployeeName: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- OtherPay: string (nullable = true)
 |-- OvertimePay: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- TotalPay: string (nullable = true)
 |-- TotalPayBenefits: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)



In [18]:
# Find the average salaries of each job position(assuming salary is the total of all pays).
salarydf.groupBy('JobTitle').agg({'TotalPay':'mean'}).show()

+--------------------+------------------+
|            JobTitle|     avg(TotalPay)|
+--------------------+------------------+
|MANAGER, UNIFIED ...|         124392.29|
|COURT ALTERNATIVE...|        58886.5175|
|       CITY ATTORNEY|         208243.12|
|HEALTH PROGRAM CO...| 80969.15729166666|
|PERFORMANCE ANALY...| 94262.28166666666|
|HEAVY EQUIPMENT O...|           92872.0|
|BATTALION CHIEF, ...| 216655.5368888889|
|ASSOCIATE PERFORM...| 56206.88909090909|
|SENIOR ESTATE INV...|          81456.45|
|              ROOFER| 68091.44888888889|
|AUTOMOTIVE MECHAN...|126170.64000000001|
|MANAGER VII - MUN...|        150641.082|
|STATION AGENT, MU...| 94206.51288461537|
|CONFIDENTIAL SECR...|          78482.26|
|CHILD SUPPORT OFF...|63523.614067796625|
|SENIOR SWIMMING I...|65161.526249999995|
| PROJECT MANAGER III|150305.20555555556|
|OPERATING ROOM NURSE|         112489.44|
|IS ENGINEER - ASS...| 91741.73000000001|
|SENIOR REAL PROPE...|      79170.671875|
+--------------------+------------

In [19]:
# Which job title has the highest full time employees?
def remove_front_char(jobtitle):
    return jobtitle.replace('(','')

remove_char_udf = udf(remove_front_char)  #defining udf for removing '('

def remove_rear_char(jobtitle):
    return jobtitle.replace(')','')

remove_rear_udf= udf(remove_rear_char)    #defining udf for removing ')'

filter1=salarydf.select(salarydf['EmployeeName'],remove_char_udf(salarydf['JobTitle'])).withColumnRenamed('remove_front_char(JobTitle)','JobTitle')
filter2=filter1.select(filter1['EmployeeName'],remove_rear_udf(filter1['JobTitle'])).withColumnRenamed('remove_rear_char(JobTitle)','JobTitle')


split_dept=filter2.select(filter2.EmployeeName,filter2.JobTitle).withColumn('JobTitle',F.explode(F.split('JobTitle',',')))    #splitting and exploding JobTitle having comma
split_dept1=split_dept.select(split_dept.EmployeeName,split_dept.JobTitle).withColumn('JobTitle',F.explode(F.split("JobTitle"," ")))   #exploding and splitting JobTitle having whitespaces


split_white_space = split_dept1.filter((split_dept1.JobTitle=='POLICE'))

final_result = split_white_space.select(split_white_space.EmployeeName).withColumn("JobTitle",lit("POLICE DEPARTMENT")).show()

+----------------+-----------------+
|    EmployeeName|         JobTitle|
+----------------+-----------------+
|    GARY JIMENEZ|POLICE DEPARTMENT|
|  ALBERT PARDINI|POLICE DEPARTMENT|
|PATRICIA JACKSON|POLICE DEPARTMENT|
| RICHARD CORRIEA|POLICE DEPARTMENT|
|    GREGORY SUHR|POLICE DEPARTMENT|
|  DENISE SCHMITT|POLICE DEPARTMENT|
|     DAVID SHINN|POLICE DEPARTMENT|
|      JOHN TURSI|POLICE DEPARTMENT|
|     JAMES BOSCH|POLICE DEPARTMENT|
|      JONES WONG|POLICE DEPARTMENT|
|    JAMES DUDLEY|POLICE DEPARTMENT|
|   KEVIN CASHMAN|POLICE DEPARTMENT|
|      ANNA BROWN|POLICE DEPARTMENT|
|     GREGORY MAR|POLICE DEPARTMENT|
|   RICHARD PARRY|POLICE DEPARTMENT|
|  THOMAS SHAWYER|POLICE DEPARTMENT|
|  ANTONIO FLORES|POLICE DEPARTMENT|
|     MARK GAMBLE|POLICE DEPARTMENT|
|   DONNA MEIXNER|POLICE DEPARTMENT|
|     JOHN MURPHY|POLICE DEPARTMENT|
+----------------+-----------------+
only showing top 20 rows



In [20]:
#Find the job titles along with the employees name and ids.
salarydf.select(salarydf['JobTitle'],salarydf['EmployeeName'],salarydf['Id']).show()

+--------------------+------------------+---+
|            JobTitle|      EmployeeName| Id|
+--------------------+------------------+---+
|GENERAL MANAGER-M...|    NATHANIEL FORD|  1|
|CAPTAIN III (POLI...|      GARY JIMENEZ|  2|
|CAPTAIN III (POLI...|    ALBERT PARDINI|  3|
|WIRE ROPE CABLE M...| CHRISTOPHER CHONG|  4|
|DEPUTY CHIEF OF D...|   PATRICK GARDNER|  5|
|ASSISTANT DEPUTY ...|    DAVID SULLIVAN|  6|
|BATTALION CHIEF, ...|         ALSON LEE|  7|
|DEPUTY DIRECTOR O...|     DAVID KUSHNER|  8|
|BATTALION CHIEF, ...|    MICHAEL MORRIS|  9|
|CHIEF OF DEPARTME...|JOANNE HAYES-WHITE| 10|
|ASSISTANT CHIEF O...|     ARTHUR KENNEY| 11|
|CAPTAIN III (POLI...|  PATRICIA JACKSON| 12|
|EXECUTIVE CONTRAC...| EDWARD HARRINGTON| 13|
|   DEPARTMENT HEAD V|       JOHN MARTIN| 14|
|BATTALION CHIEF, ...|    DAVID FRANKLIN| 15|
|COMMANDER III, (P...|   RICHARD CORRIEA| 16|
|   DEPARTMENT HEAD V|          AMY HART| 17|
|CAPTAIN, EMERGENC...|    SEBASTIAN WONG| 18|
|BATTALION CHIEF, ...|        MART

In [21]:
#List out the names and positions of employees whose total pay is greater than 180000.


salary_filter = salarydf.filter(salarydf['TotalPay']>180000)
result = salary_filter.select(salary_filter['EmployeeName'],salary_filter['JobTitle'],salary_filter['TotalPay']).show()



+------------------+--------------------+---------+
|      EmployeeName|            JobTitle| TotalPay|
+------------------+--------------------+---------+
|    NATHANIEL FORD|GENERAL MANAGER-M...|567595.43|
|      GARY JIMENEZ|CAPTAIN III (POLI...|538909.28|
|    ALBERT PARDINI|CAPTAIN III (POLI...|335279.91|
| CHRISTOPHER CHONG|WIRE ROPE CABLE M...|332343.61|
|   PATRICK GARDNER|DEPUTY CHIEF OF D...|326373.19|
|    DAVID SULLIVAN|ASSISTANT DEPUTY ...|316285.74|
|         ALSON LEE|BATTALION CHIEF, ...|315981.05|
|     DAVID KUSHNER|DEPUTY DIRECTOR O...|307899.46|
|    MICHAEL MORRIS|BATTALION CHIEF, ...|303427.55|
|JOANNE HAYES-WHITE|CHIEF OF DEPARTME...|302377.73|
|     ARTHUR KENNEY|ASSISTANT CHIEF O...|299494.17|
|  PATRICIA JACKSON|CAPTAIN III (POLI...|297608.92|
| EDWARD HARRINGTON|EXECUTIVE CONTRAC...|294580.02|
|       JOHN MARTIN|   DEPARTMENT HEAD V|292671.62|
|    DAVID FRANKLIN|BATTALION CHIEF, ...|286347.05|
|   RICHARD CORRIEA|COMMANDER III, (P...|286213.86|
|          A

In [22]:

# Q7 List the names and ids of employees who have never done overtime.

overtime_filter = salarydf.filter(salarydf['OvertimePay']==0)
result = overtime_filter.select(overtime_filter['EmployeeName'],overtime_filter['Id']).show()

+--------------------+---+
|        EmployeeName| Id|
+--------------------+---+
|      NATHANIEL FORD|  1|
|       DAVID KUSHNER|  8|
|  JOANNE HAYES-WHITE| 10|
|   EDWARD HARRINGTON| 13|
|         JOHN MARTIN| 14|
|            AMY HART| 17|
|          VENUS AZAR| 21|
|        GREGORY SUHR| 26|
|      DENISE SCHMITT| 29|
|       MONICA FIELDS| 30|
|     HARLAN KELLY-JR| 31|
|         DAVID SHINN| 32|
|         GARY AMELIO| 33|
|        SUSAN CURRIN| 37|
|SHARON MCCOLE WICHER| 45|
|           EDWIN LEE| 46|
|        TRENT RHORER| 48|
|        JAMES DUDLEY| 49|
|      BARBARA GARCIA| 52|
|       KEVIN CASHMAN| 56|
+--------------------+---+
only showing top 20 rows

